- Ricardo Méndez 21289
- Sara Echeverría 21371
- Melissa Pérez 21385
- Francisco Castillo 21562

# Task 1

#### ¿Por qué el modelo de Naive Bayes se le considera “naive”?
La traducción del inglés al español de la palabra "naive" es ingenuo por lo que al modelo se le considera de esa forma debido a que sus variables predictorias son independientes entre sí. Donde básicamente describe que una característica de un conjunto de datos no está relacionada con otra de ellas. En otras palabras, es por su simplicidad de la independencia condicional entre las características o atributos que describen los datos. (Roman, 2019)

#### Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo? 


### ¿Qué tipo de ensemble learning es este modelo (Random Forest)?

### ¿Cuál es la idea general detrás de Random Forest?

### ¿Por qué se busca baja correlación entre los árboles de Random Forest?

### Referencias
- Roman, V., (2019). Algoritmos Naive Bayes: Fundamentos e Implementación. https://medium.com/datos-y-ciencia/algoritmos-naive-bayes-fudamentos-e-implementaci%C3%B3n-4bcb24b307f 

# Task 2 Naive Bayes: Clasificador de Mensajes Ham/Spam

# Task 2.1

# Task 2.2

# Task 2.3

# Task 2.4

# Task 3 Clasificación de Partidas de League of Legends

# Task 3.1 Support Vector Machines

# Task 3.2 Árboles de Decisión

# Task 3.3 Comparación